In [0]:
import os
import numpy as np
import pandas as pd
import random 
import copy
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.metrics import mean_absolute_error as MAE
from sklearn import linear_model as lm
from keras.models import Sequential
from keras.layers import Activation, Dense,Bidirectional,Dropout,LSTM
from itertools import chain

Using TensorFlow backend.


In [0]:
 original_data= pd.read_excel('YES_BANK.xlsx',
header=None,
index_col=False,
keep_default_na=True,names=['Date','Open','High','Low','Close','Volume']
)


In [0]:
original_data

,Date,Open,High,Low,Close,Volume
0,2020-03-09,17.00,22.80,16.20,21.25,69542810
1,2020-03-06,33.20,33.20,5.55,16.20,106336244
2,2020-03-05,29.40,37.90,28.05,36.85,43386506
3,2020-03-04,31.40,31.70,28.65,29.30,10055149
4,2020-03-03,32.80,33.30,30.60,31.25,10299879
...,...,...,...,...,...,...
3627,2005-07-18,12.56,13.13,12.47,12.85,30424025
3628,2005-07-15,11.56,12.88,11.25,12.35,36693185
3629,2005-07-14,11.88,11.96,11.40,11.46,6810720
3630,2005-07-13,12.29,12.47,11.48,11.70,28784240


In [0]:
original_data=original_data.iloc[::-1]
list_=original_data.values.tolist()
original_data=pd.DataFrame(list_)
#original_data
original_data.columns=["Date"	,"Open",	"High",	"Low",	"Close",	"Volume"]
original_data['Date'] = pd.to_datetime(original_data['Date'])
original_data.head()

,Date,Open,High,Low,Close,Volume
0,2005-07-12,13.00,14.00,12.03,12.16,130706350
1,2005-07-13,12.29,12.47,11.48,11.70,28784240
2,2005-07-14,11.88,11.96,11.40,11.46,6810720
3,2005-07-15,11.56,12.88,11.25,12.35,36693185
4,2005-07-18,12.56,13.13,12.47,12.85,30424025


In [0]:
def datasetpreparation(original_data):
  data_split = int(len(original_data)*0.8)
  data1 = original_data.drop(['Date','Volume'],axis = 1)
  data1.head()
  data1_input = []
  for i in range(len(data1) - 4):
    list1 = data1[i:i+5].values.tolist()
    data1_input.append(list1)
  #print(np.array(data4_input).shape)
  #(data4_input)
  data1_output = data1.ix[5:,:]
  #data4_output
  train1_input = np.array(data1_input[0:data_split])
  train1_output = np.array(data1_output[0:data_split])
  print(train1_input.shape)
  print(train1_output.shape)
  test1_input = np.array(data1_input[data_split:])
  test1_output = np.array(data1_output[data_split:])
  print(test1_input.shape)
  print(test1_output.shape)
  return train1_input,train1_output,test1_input,test1_output

In [0]:
def LSTMmodel():
  model1 = Sequential()
  model1.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(5, 4)))
  model1.add(Dropout(0.25))
  model1.add(Dense(4))
  model1.compile(optimizer='adam', loss='mae')
  return model1

In [0]:
def modeltrain(train1_input,train1_output,model1):
  #train_input,train_output,test1_input,test1_output = datasetpreparation(original_data)
  #model = LSTMmodel 
  history = model1.fit(train1_input, train1_output, epochs=600, verbose=1)
  model1.save('YES_BANK.h5')

In [0]:
def evaluateval(model1,train1_input,train1_output):
  #train_input,train_output,test1_input,test1_output = datasetpreparation(original_data)
  predict = model1.predict(train1_input)
  predict1 = pd.DataFrame(predict)
  output = pd.DataFrame(train1_output)
  for i in range(4):
    plt.plot(predict1.ix[:,i])
    plt.plot(output.ix[:,i])
    plt.show()

In [0]:
def test(model1,train1_input,train1_output,test1_input,test1_output,i):
  input = test1_input[i].reshape(1,5,4)
  train1_input = np.concatenate((train1_input,input),axis = 0)
  train1_input = train1_input[-20:]
  output = test1_output[i].reshape(1,4)
  train1_output = np.concatenate((train1_output,output),axis = 0)
  train1_output = train1_output[-20:]
  prediction = model1.predict(input)
  return prediction,train1_input,train1_output

In [0]:
def feedback(model1,train1_input,train1_output,test1_input,i):
  model1.fit(train1_input, train1_output, epochs=10, verbose=1)
  prediction = model1.predict(test1_input[i].reshape(1,5,4))
  return prediction

In [0]:
from keras.models import load_model
  def predict(original_data):
  train1_input,train1_output,test1_input,test1_output = datasetpreparation(original_data)
  model1 = LSTMmodel()
  model1 = load_model('YES_BANK.h5')
  pred_list = []
  pred_list_feedback = []
  for i in range(len(test1_input)):
   pred,train1_input,train1_output = test(model1,train1_input,train1_output,test1_input,test1_output,i)
   
   pred=(pred.tolist())
   pred = list(chain.from_iterable(pred))
   pred_list.append(pred)

   prediction = feedback(model1,train1_input,train1_output,test1_input,i)
   prediction=(prediction.tolist())
   prediction = list(chain.from_iterable(prediction))
   pred_list_feedback.append(prediction)
   #print(pred)
   #print(prediction)
   #print(test1_output[i])
   #print(pred_list)
   #print(pred_list_feedback)  
  return pred_list, pred_list_feedback 


In [0]:
def runprograme(original_data):
  train1_input,train1_output,test1_input,test1_output = datasetpreparation(original_data)
  model1 = LSTMmodel()
  modeltrain(train1_input,train1_output,model1)
  model1 = load_model('YES_BANK.h5')
  evaluateval(model1,train1_input,train1_output)
  pred_list,pred_list_feedback = predict(original_data)
  return pred_list, pred_list_feedback 


In [0]:
pred_list, pred_list_feedback = runprograme(original_data)


In [0]:
#predict = model1.predict(test1_input)
  predict1 = pred_list_feedback
  output = pd.DataFrame(test1_output)
  for i in range(4):
    plt.plot(predict1.ix[:,i])
    plt.plot(output.ix[:,i])
    plt.show()